In [ ]:
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

In [ ]:
# browser = webdriver.Chrome(ChromeDriverManager().install()) # use this to install chromium
browser = webdriver.Chrome() # use this line if chromedriver is in PATH

In [ ]:
def one_letter_at_a_time(delay: float):
    # version 1: send leter by letter
    words = browser.find_element(By.ID, "words").find_elements(By.CLASS_NAME, "word")
    try:
        while len(words) != 0:
            active_index = [i for i in range(len(words)) if 'active' in words[i].get_attribute('class')][0]
            words = words[active_index:]
            all_letters = ""
            for word in words:
                letters = word.find_elements(By.TAG_NAME, "letter")
                for letter in letters:
                    all_letters += letter.text
                all_letters += " "
            for letter in all_letters:
                ActionChains(browser).send_keys(letter).perform()
                time.sleep(delay)
            words = browser.find_element(By.ID, "words").find_elements(By.CLASS_NAME, "word")
    except Exception as e:
        print(e)
    print("Game Over")

In [ ]:
def all_letters_at_a_time(delay: float):
    # version 2: send all letters at once
    words = browser.find_element(By.ID, "words").find_elements(By.CLASS_NAME, "word")
    try:
        while len(words) != 0:
            active_index = [i for i in range(len(words)) if 'active' in words[i].get_attribute('class')][0]
            words = words[active_index:]
            all_letters = ""
            for word in words:
                letters = word.find_elements(By.TAG_NAME, "letter")
                for letter in letters:
                    all_letters += letter.text
                all_letters += " "
            ActionChains(browser).send_keys(all_letters).perform()
            time.sleep(delay)
            words = browser.find_element(By.ID, "words").find_elements(By.CLASS_NAME, "word")
    except Exception as e:
        print(e)
    print("Game Over")


In [ ]:
def one_word_at_a_time(delay: float):
    # version 3: detect one active word at a time and input (fastest way)
    try:
        while len(browser.find_elements(By.CLASS_NAME, "word")) != 0:
            ActionChains(browser).send_keys([letter.text for letter in browser.find_element(By.CSS_SELECTOR, ".word.active").find_elements(By.TAG_NAME, "letter")] + [' ']).perform()
            time.sleep(delay)
    except Exception as e:
        print(e)
    print("Game Over")

In [ ]:
def one_word_letter_at_a_time(wpm=500):
    import random, time
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.action_chains import ActionChains

    delay = 1 / wpm # Accurate delay per character to match WPM
    print(f"Target WPM {wpm} → Delay per char: {delay:.4f} sec")

    try:
        start_time = time.time()

        while len(browser.find_elements(By.CLASS_NAME, "word")) != 0:
            active_word = browser.find_element(By.CSS_SELECTOR, ".word.active")
            letters = [letter.text for letter in active_word.find_elements(By.TAG_NAME, "letter")] + [' ']

            for letter in letters:
                elapsed = time.time() - start_time

                # Dynamic typing speed to simulate human rhythm
                if elapsed < 4:
                    varied_delay = random.uniform(delay * 0.3, delay * 0.6)
                elif elapsed < 10:
                    varied_delay = random.uniform(delay * 0.7, delay * 1.1)
                else:
                    varied_delay = random.uniform(delay * 0.95, delay * 1.1)

                # Occasional pause patterns (human-like)
                r = random.random()
                if r < 0.005:
                    time.sleep(1.0)      # deep pause
                elif r < 0.01:
                    time.sleep(0.5)      # slight hesitation
                elif r < 0.015:
                    time.sleep(0.25)
                elif r < 0.02:
                    time.sleep(0.125)
                elif r < 0.025:
                    time.sleep(0.0231)

                # Slow down on awkward keys
                if len(letter) > 1 or letter.lower() in ['p', 'y', 'g']:
                    time.sleep(0.025)

                # Simulate realistic key press + hold + release
                ActionChains(browser).key_down(letter).perform()
                time.sleep(random.uniform(0.040, 0.050))  # key hold: 40–50ms
                ActionChains(browser).key_up(letter).perform()
                time.sleep(varied_delay)

                # 🔁 2% chance: Make a typo and fix it
                if random.random() < 0.02 and letter.isalpha():
                    wrong_letter = random.choice("abcdefghijklmnopqrstuvwxyz")
                    ActionChains(browser).send_keys(wrong_letter).perform()
                    time.sleep(varied_delay + 0.04)
                    ActionChains(browser).send_keys('\b').perform()
                    time.sleep(0.04)

                # 🔁 1% chance: Typo not corrected
                if random.random() < 0.01 and letter.isalpha():
                    wrong_letter = random.choice("abcdefghijklmnopqrstuvwxyz")
                    ActionChains(browser).send_keys(wrong_letter).perform()
                    time.sleep(varied_delay)
                    continue  # skip correct letter to simulate typo

    except Exception as e:
        print("⚠️ Exception:", e)

    print("✅ Game Over (Humanized Typing Finished)")


In [ ]:
url = 'https://monkeytype.com/'
browser.get(url)

In [ ]:
try:
    accept_all_btn = browser.find_element(By.CSS_SELECTOR, ".button.active.acceptAll")
    accept_all_btn.click()
except Exception:
    pass

In [ ]:
# focus on page
browser.find_element(By.CSS_SELECTOR, ".word.active").click()

In [ ]:
time.sleep(5)
delay = 0.02 # 0.0185
# uncomment one of the following lines to test different methods
one_letter_at_a_time(delay)
# all_letters_at_a_time(delay)
# one_word_at_a_time(delay)
# one_word_letter_at_a_time(delay)
# type_human_like

In [ ]:
wpm = browser.find_element(By.CSS_SELECTOR, ".group.wpm").find_element(By.CLASS_NAME, "bottom").text
acc = browser.find_element(By.CSS_SELECTOR, ".group.acc").find_element(By.CLASS_NAME, "bottom").text
consistency = browser.find_element(By.CSS_SELECTOR, ".group.flat.consistency").find_element(By.CLASS_NAME, "bottom").text
print("wpm: " + wpm)
print("accuracy: " + acc)
print("consistency: " + consistency)